### 1) FACT_ORDERS

In [0]:
from pyspark.sql.functions import sum,avg,col

client_id     = dbutils.secrets.get(scope="kv-olist", key="secret-client-id")
tenant_id     = dbutils.secrets.get(scope="kv-olist", key="secret-tenant-id")
client_secret = dbutils.secrets.get(scope="kv-olist", key="secret-client-mdp")

storage_account = "oliststorageaccountbth74"
container_name = "data"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

df_orders=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_orders_dataset")

df_payments=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_order_payments_dataset")

df_reviews=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_order_reviews_dataset")

df_pivot_payments=df_payments.groupBy("order_id").pivot("payment_type").agg(sum("payment_value")).fillna(0)

df_reviews_filtered=df_reviews.groupBy("order_id").agg(avg("review_score").alias("avg_review_score"))

df_orders_joined=df_orders.join(df_pivot_payments, on="order_id", how="left").join(df_reviews_filtered,on="order_id",how="left")

df_orders_joined=df_orders_joined.withColumn("total_order",col("boleto")+col("credit_card")+col("debit_card")+col("voucher"))


df_orders_joined.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/gold/olist_fact_orders")


Databricks data profile. Run in Databricks to view.

### 2) FACT_ORDER_ITEMS

In [0]:
from pyspark.sql.functions import col

client_id     = dbutils.secrets.get(scope="kv-olist", key="secret-client-id")
tenant_id     = dbutils.secrets.get(scope="kv-olist", key="secret-tenant-id")
client_secret = dbutils.secrets.get(scope="kv-olist", key="secret-client-mdp")

storage_account = "oliststorageaccountbth74"
container_name = "data"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

df_order_items=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_order_items_dataset")

df_orders=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_orders_dataset")

df_customerId=df_orders.select("order_id","customer_id","order_purchase_timestamp")

df_order_items_joined=df_order_items.join(df_customerId,on="order_id",how="inner")

df_order_items_joined=df_order_items_joined.withColumn("total_order",col("price")+col("freight_value"))

df_order_items_joined.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/gold/olist_fact_order_items")

Databricks data profile. Run in Databricks to view.

### 3) DIM_PRODUCTS 

In [0]:
from pyspark.sql.functions import coalesce,col,lit

client_id     = dbutils.secrets.get(scope="kv-olist", key="secret-client-id")
tenant_id     = dbutils.secrets.get(scope="kv-olist", key="secret-tenant-id")
client_secret = dbutils.secrets.get(scope="kv-olist", key="secret-client-mdp")

storage_account = "oliststorageaccountbth74"
container_name = "data"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


df_products=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_products_dataset")

df_translations=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/product_category_name_translation")

df_products_translated=df_products.join(df_translations,on="product_category_name",how="left")

df_products_translated=df_products_translated.withColumn(
    "product_category_name_english",
    coalesce(col("product_category_name_english"),col("product_category_name"),lit("unknown")))

df_products_translated=df_products_translated.drop("product_category_name")

df_products_translated=df_products_translated.withColumnRenamed("product_category_name_english","product_category_name")

df_products_translated.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/gold/olist_dim_products")

Databricks data profile. Run in Databricks to view.

### 4) DIM_CUSTOMERS

In [0]:
client_id     = dbutils.secrets.get(scope="kv-olist", key="secret-client-id")
tenant_id     = dbutils.secrets.get(scope="kv-olist", key="secret-tenant-id")
client_secret = dbutils.secrets.get(scope="kv-olist", key="secret-client-mdp")

storage_account = "oliststorageaccountbth74"
container_name = "data"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

df_customers=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_customers_dataset")

df_customers.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/gold/olist_dim_customers")


Databricks data profile. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

### 5) DIM_SELLERS

In [0]:
from pyspark.sql.functions import col, trim, count
from pyspark.sql.window import Window

client_id     = dbutils.secrets.get(scope="kv-olist", key="secret-client-id")
tenant_id     = dbutils.secrets.get(scope="kv-olist", key="secret-tenant-id")
client_secret = dbutils.secrets.get(scope="kv-olist", key="secret-client-mdp")

storage_account = "oliststorageaccountbth74"
container_name = "data"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


df_sellers=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/olist_sellers_dataset")

df_sellers=df_sellers.withColumnRenamed("seller_zip_code_prefix","seller_zip_code")

df_sellers=df_sellers.withColumn("seller_id", trim(col("seller_id")))
df_sellers_unique=df_sellers.dropDuplicates(["seller_id"])

df_sellers_unique.write.format("delta").mode("overwrite").save(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/gold/olist_dim_sellers")



Databricks data profile. Run in Databricks to view.